In [132]:
import pandas as pd
import tensorflow as tf

In [133]:
df = pd.read_csv('/content/fakeNews.csv', on_bad_lines='skip', engine='python')

In [134]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [135]:
df.isnull().sum()

,0
id,0
title,118
author,441
text,11
label,0


In [136]:
# Instead of fillNa use DropNa
df = df.dropna()

In [137]:
df.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [138]:
x = df.drop('label', axis=1)
y = df['label']

In [139]:
x.shape

(3872, 4)

In [140]:
y.shape

(3872,)

In [141]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [142]:
vocabSize = 10000

In [143]:
# One hot Representation
messages = x.copy()
messages.reset_index(inplace=True)   # Reseting Index Because Of DropNA

In [144]:
# Data Preprocessing
import nltk
import re  # Regular Expressions
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [145]:
!pip install PorterStemmer

In [146]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) #Substituting ' ' everywhere apart from a-z and A-Z in title of every index
  review = review.lower() # Lowering all Words
  review = review.split() # Splitting helps in removing stopwords
  # This review is list of words in that  specific sentence
  review = [stemmer.stem(word) for word in review if not word in stopwords.words('english')]  # Stemming Those Words that are unique and not present in stopwords are stemmed and those stemmed words are stored in review
  review = ' '.join(review)
  corpus.append(review)

In [147]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [148]:
# One Hot Representation
oneHotList = []
for word in corpus:
  oneHotList.append(one_hot(word, vocabSize))

In [149]:
oneHotList

[[1774, 4131, 5799, 8616, 8613, 4969, 1568, 8263, 4972, 4596],
 [9997, 3532, 9977, 3711, 124, 1293, 2084],
 [2462, 8151, 7107, 5048],
 [4061, 8790, 2013, 2853, 3832, 383],
 [2201, 124, 6727, 4128, 2500, 4818, 124, 7582, 4704, 2410],
 [3736,
  4452,
  8018,
  946,
  1644,
  8684,
  9075,
  4360,
  331,
  6027,
  3755,
  5270,
  6153,
  2500,
  2084],
 [9122, 3637, 3875, 165, 8323, 878, 3240, 4267, 1750, 5844, 8905],
 [5472, 726, 414, 1335, 5130, 5269, 8684, 8327, 1750, 5844, 8905],
 [257, 7954, 7251, 5401, 9391, 3331, 1636, 3760, 8684, 3843],
 [503, 9746, 4630, 4171, 4738, 9402, 852, 4782],
 [9, 7016, 3150, 9251, 7364, 2817, 9896, 8767, 8213, 8279, 8258],
 [2853, 9484, 8613, 3331, 8684, 5130],
 [9349, 9773, 3915, 6364, 8099, 5744, 1418, 5473, 2198],
 [7473, 979, 5471, 2616, 705, 6994, 6768, 1750, 5844, 8905],
 [9442, 1503, 1180, 2095, 6586, 1750, 5844, 8905],
 [6974, 1420, 8512, 7988, 8040, 4901, 5149, 3475, 603, 6373],
 [963, 4728, 3532],
 [4901, 3160, 2510, 1218, 8684, 2013, 787, 2084

In [150]:
# Padding in Front of each sentence to make all sentence of Equal Length
sentenceLength = 25
embeddedDocs = pad_sequences(oneHotList, padding='pre', maxlen=sentenceLength)

In [151]:
embeddedDocs

array([[   0,    0,    0, ..., 8263, 4972, 4596],
       [   0,    0,    0, ...,  124, 1293, 2084],
       [   0,    0,    0, ..., 8151, 7107, 5048],
       ...,
       [   0,    0,    0, ..., 3151, 2235, 2084],
       [   0,    0,    0, ..., 1750, 5844, 8905],
       [   0,    0,    0, ..., 8684,  945, 1531]], dtype=int32)

In [152]:
# Model Creation

# Previous Old Syntax -->UserWarning: Argument input_length is deprecated. Just remove it. And a blank summary because the input shape wasn't fully set.
# embeddingVectorFeatures = 40  #Dimensions
# model = Sequential()
# model.add(Embedding(vocabSize, embeddingVectorFeatures, input_length=sentenceLength))
# model.add(LSTM(100))  # 1 LSTM layer with 100 Neurons
# model.add(Dense(1, activation='sigmoid'))  # Dense Layer Because Of Classifications
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())

# Correct Code
embeddingVectorFeatures = 40
model = Sequential()  # Sequential Model
model.add(Embedding(input_dim=vocabSize, output_dim=embeddingVectorFeatures, input_shape=(sentenceLength,)))  # Embedding Layer
model.add(LSTM(100)) # LSTM Layer
model.add(Dense(1, activation='sigmoid'))  # Dense Layer For Classification
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Model Compilation
model.summary()


# Why This Works
# input_shape=(sentenceLength,) properly initializes the input dimensions.
# input_length= is kept in some old tutorials but no longer needed in newer TensorFlow/Keras versions.

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 25, 40)         │       400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 456,501 (1.74 MB)

 Trainable params: 456,501 (1.74 MB)

 Non-trainable params: 0 (0.00 B)

In [153]:
# Converting Padded Embedding Docs In 1D array For Model Training
import numpy as np
xFinal = np.array(embeddedDocs)
yFinal = np.array(y)

In [154]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(xFinal, yFinal, test_size=0.33, random_state=42)

In [155]:
# Model 1 Training
model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=64)

Epoch 1/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7226 - loss: 0.6223 - val_accuracy: 0.8623 - val_loss: 0.3487
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9000 - loss: 0.2758 - val_accuracy: 0.8889 - val_loss: 0.2592
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9486 - loss: 0.1598 - val_accuracy: 0.8764 - val_loss: 0.2464
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9827 - loss: 0.0846 - val_accuracy: 0.8920 - val_loss: 0.3378
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9879 - loss: 0.0416 - val_accuracy: 0.8764 - val_loss: 0.3379
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9967 - loss: 0.0239 - val_accuracy: 0.8951 - val_loss: 0.3419
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9972 - loss: 0.0131 - val_accuracy: 0.8944 - val_loss: 0.3782
Epoch 8/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9978 - loss: 0.0108 - val_accuracy: 0.8975 - val

In [156]:
# Predictions of Model 1
predictions = model.predict(xTest)
predictions = (predictions > 0.5).astype(int)  # Convert probabilities to binary labels


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [157]:
# Perforamance of Model 1
from sklearn.metrics import accuracy_score
accuracy_score(yTest, predictions)

0.8974960876369327

In [158]:
# Perforamance of Model 1
from sklearn.metrics  import confusion_matrix
confusion_matrix(yTest, predictions)

array([[643,  88],
       [ 43, 504]])

In [159]:
# Adding Dropout(Hyperparameter Optimisation)
from tensorflow.keras.layers import Dropout
embeddingVectorFeatures = 40
model2 = Sequential()
model2.add(Embedding(input_dim=vocabSize, output_dim=embeddingVectorFeatures, input_shape=(sentenceLength,)))
model2.add(Dropout(0.3))
model2.add(LSTM(100))
model2.add(Dropout(0.3))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 25, 40)         │       400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 25, 40)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 456,501 (1.74 MB)

 Trainable params: 456,501 (1.74 MB)

 Non-trainable params: 0 (0.00 B)

In [160]:
# Model 2 With Dropout Layers Training
model2.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=64)

Epoch 1/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.6421 - loss: 0.6327 - val_accuracy: 0.8451 - val_loss: 0.3553
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8874 - loss: 0.2924 - val_accuracy: 0.8951 - val_loss: 0.2618
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9397 - loss: 0.1719 - val_accuracy: 0.9014 - val_loss: 0.2241
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9723 - loss: 0.0930 - val_accuracy: 0.8873 - val_loss: 0.2653
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9876 - loss: 0.0487 - val_accuracy: 0.8803 - val_loss: 0.3121
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9932 - loss: 0.0311 - val_accuracy: 0.8920 - val_loss: 0.3116
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9960 - loss: 0.0206 - val_accuracy: 0.8959 - val_loss: 0.3134
Epoch 8/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9989 - loss: 0.0105 - val_accuracy: 0.8975 - v

In [161]:
# Predictions of Model 2
predictions2 = model2.predict(xTest)
predictions2 = (predictions > 0.5).astype(int)  # Convert probabilities to binary labels

40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


In [162]:
# Perforamance of Model 2
from sklearn.metrics import accuracy_score
accuracy_score(yTest, predictions2)

0.8974960876369327

In [163]:
# Perforamance of Model 2
from sklearn.metrics  import confusion_matrix
confusion_matrix(yTest, predictions2)

array([[643,  88],
       [ 43, 504]])